In [1]:
import datetime as dt
import pandas as pd
import os
import requests
import cv2

In [2]:
download_dir = '/data2/elilouis/sublimationofsnow/eol_photos/'

In [8]:
start_date = '20230501'
end_date = '20230525'
DATE_FORMAT_STR = '%Y%m%d'

In [9]:
def get_url(date):
    return \
    f"https://archive.eol.ucar.edu/docs/isf/projects/SOS/isfs/qcdata/images/{date}/surface.ISFS.{date}0002.photo.jpg"

def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


In [10]:
datelist = pd.date_range(
    dt.datetime.strptime(start_date, DATE_FORMAT_STR),
    dt.datetime.strptime(end_date, DATE_FORMAT_STR),
    freq='d'
).strftime(DATE_FORMAT_STR).tolist()


In [11]:
for date in datelist:
    print(f"downloading: {date}")
    download(
        get_url(date),
        download_dir
    )
    assert os.path.isfile(
        os.path.join(download_dir, f"surface.ISFS.{date}0002.photo.jpg")
    )

downloading: 20230501
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305010002.photo.jpg
downloading: 20230502
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305020002.photo.jpg
downloading: 20230503
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305030002.photo.jpg
downloading: 20230504
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305040002.photo.jpg
downloading: 20230505
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305050002.photo.jpg
downloading: 20230506
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305060002.photo.jpg
downloading: 20230507
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305070002.photo.jpg
downloading: 20230508
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305080002.photo.jpg
downloading: 20230509
saving to /data2/elilouis/sublimationofsnow/eol_photos/surface.ISFS.202305

In [20]:

import os

image_folder = download_dir
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()


In [25]:
ls | grep v

disdro_daily_max_precip_date_df.csv
evaporation_or_sublimation.ipynb
low_level_jet_analysis.ipynb
make_timelapse_video.ipynb
monthly_planar_fits.csv
Stossel_box_weights - Sheet1.csv
tidy_df_30Min_20221201_20230501_planar_fit.csv
tidy_df_30Min.csv
tidy_df_30Min_noplanar_fit.csv
tidy_df_30Min_planar_fit.csv
video.avi
